# Development

Agreement and policy: https://developer.twitter.com/en/developer-terms/agreement-and-policy

In [1]:
import os
os.chdir('..')

import tweepy
import requests
import psycopg2
import pandas as pd

from authenticators import (
    TwitterAuthenticator, RDSAuthenticator
)

twitter = TwitterAuthenticator()
client = tweepy.Client(bearer_token=twitter.bearer_token)

rds = RDSAuthenticator()
connection = psycopg2.connect(
    host = rds.host,
    port = rds.port,
    database= rds.database,
    user = rds.user,
    password = rds.password
)
connection.autocommit = True
cursor = connection.cursor()

¿En qué necesito pensar?

- Conectarme a RDS
- Esquema para extraer datos uniformemente durante los últimos 7 días
- Establecer qué políticos tomar en cuenta. Scrape the list from here: https://es.wikipedia.org/wiki/Elecciones_generales_de_Guatemala_de_2023
- ¿Guardar los datos en una base de datos SQL local?


## Get tweets

In [ ]:
def get_tweets(query: str, start_time: str, end_time: str, max_results: int) -> requests.Response:
    query = query = f"{query} -is:retweet -is:reply"
    tweets = client.search_recent_tweets(
        query = query,
        start_time = start_time,
        end_time = end_time,
        max_results = max_results,
        tweet_fields = [
            "id", "author_id", "created_at", "text", 
            "public_metrics", "possibly_sensitive", "lang"
        ],
        user_fields = [
            "id", "username", "name", "location", "created_at", "description", 
            "profile_image_url", "verified", "public_metrics"
        ],
        expansions = [
            "author_id", "referenced_tweets.id"
        ]
    )
    return tweets

## Process them into DataFrames

In [ ]:
class TwitterDataCleaner:
    def __init__(self, tweets: requests.Response):
        self.tweets = tweets
        self.df = None

    def clean(self) -> pd.DataFrame:
        self.extract_tweet_data()
        self.extract_user_data()

        tweets_df, users_df = self.segregate_data()
        return tweets_df, users_df

    def extract_tweet_data(self):
        tweet_data = []
        for tweet in self.tweets.data:
            tweet_dict = {key: getattr(tweet, key) for key in tweet.data.keys()}
            public_metrics = tweet_dict.pop('public_metrics')
            tweet_dict.update(public_metrics)
            tweet_data.append(tweet_dict)

        self.df = pd.DataFrame(tweet_data)

    def extract_user_data(self):
        users = {user.id: user for user in self.tweets.includes['users']}
        for key, user in users.items():
            user_data = {f"user_{key}": getattr(user, key) for key in user.data.keys()}
            public_metrics_user = user_data.pop('user_public_metrics')
            user_data.update({f"user_{k}": v for k, v in public_metrics_user.items()})
            users[key] = user_data

        self.df['user_data'] = self.df['author_id'].apply(lambda x: users[x])

        user_columns = pd.json_normalize(self.df['user_data']).columns
        for col in user_columns:
            self.df[col] = self.df['user_data'].apply(lambda x: x.get(col, None))

        self.df = self.df.drop(columns = ['user_data'])

    def segregate_data(self):
        tweets_df = self.df[[
            "id", "author_id", "created_at", "text", "possibly_sensitive", "retweet_count",
            "reply_count", "like_count", "quote_count", "impression_count", "lang"
        ]]

        users_df = self.df[[
            "user_id", "user_username", "user_name", "user_location", "user_created_at",
            "user_description", "user_profile_image_url", "user_verified",
            "user_followers_count", "user_following_count", "user_tweet_count", "user_listed_count"
        ]]

        return tweets_df, users_df

## Store them in a RDS instance

In [ ]:
cursor.close()
connection.close()

In [ ]:
# Create a sample DataFrame
data = {
    'column1': [1, 2, 3],
    'column2': ['A', 'B', 'C']
}
df = pd.DataFrame(data)

# Upload the DataFrame to your PostgreSQL instance
# Replace "your_table_name" with the desired table name
df.to_sql('your_table_name', engine, if_exists='replace', index=False)


## Loop through multiple days and candidates

¿Debería hacer append a un dataframe o cargar los datos a una base de datos SQL?

In [ ]:
# candidates = [
#     'JoeBiden', 'KamalaHarris', 'BernieSanders'
# ]

# for candidate in candidates:
#     query = candidate

#     for date in dates:
#         start_time = '2021-01-01T00:00:00Z'
#         end_time = '2021-01-02T00:00:00Z'
#         max_results = 10

#         tweets_call = get_tweets(query, start_time, end_time, max_results)
#         cleaner = TwitterDataCleaner(tweets_call)
#         tweets, users = cleaner.clean()
        
#         ingest_tweets(tweets, users, path)

## Example usage

In [ ]:
query = 'jimmy morales'
start_time = "2023-04-25T00:00:00Z" 
end_time = "2023-04-28T00:00:00Z" 
max_results = 10

tweets_call = get_tweets(
    query=query,
    start_time=start_time,
    end_time=end_time,
    max_results=max_results,
)

data_cleaner = TwitterDataCleaner(tweets_call)
tweets, users = data_cleaner.clean()

In [ ]:
tweets.head()

In [ ]:
users.head()